In [89]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
import numpy as np
import pandas as pd
import matplotlib as plt
import sys 
import importlib
import re
URL = {'Data analyst':"https://sg.indeed.com/jobs?as_and=data+analyst&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&radius=0&l=Singapore&fromage=any&limit=10&sort=&psf=advsrch&from=advancedsearch",
       'Data scientist': 'https://sg.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&radius=0&l=Singapore&fromage=any&limit=10&sort=&psf=advsrch&from=advancedsearch'
       }
#'Data engineer': 'https://hk.indeed.com/jobs?q=data%20engineer&l=Hong%20Kong&radius=0&vjk=7fd03b6ed658179e'
href = {}
for category ,URL in URL.items():
    html = requests.get(URL)
    soup = BeautifulSoup(html.text, "html.parser")    
    cnt = 0

    while True:
        # find all job postings in each page
        for i in soup.find_all('h2',class_='title'):        
            try:
                href['https://sg.indeed.com' + i.a['href']] = category

            except:
                np.nan

        # go to next page
        cnt += 1 # number of times that have proceeded to next page so far
        pg_num = int(soup.find(class_='pagination').find_all('a',attrs = {'href':True})[-1]['href'].split('=')[-1])/10
        # to get the page number from url

        if cnt == pg_num:
            next_pg = soup.find(class_='pagination').find_all('a',attrs = {'href':True})[-1]['href']
            URL2 = 'https://sg.indeed.com'+ next_pg
            html = requests.get(URL2)
            soup = BeautifulSoup(html.text,'lxml')
        else:
            break

In [90]:
len(href.keys())

1152

In [91]:
# get title, company name, salary, job requirement, soft skills, technical skills, industry 
title = []
company = []
ftpt= []
location = []
issued_time = []
description = []
salary = []
review = []
review_num = []

cnt = 0
for i in href.keys():
    # go to each link one by one
    indi_html = requests.get(i)
    indi_soup = BeautifulSoup(indi_html.text,'html.parser')
    
    # find job title
    try:
        indi_title = indi_soup.find('h3',class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").text       
    except:
        indi_title = np.nan
    
    # find company name
    try:
        indi_company = indi_soup.find('div',class_="icl-u-lg-mr--sm icl-u-xs-mr--xs").text     
    except:
        indi_company = np.nan
    # find job type    
    try:
        #to find ptft keywords by joining all results as string
        temp = ''.join([str(i) for i in indi_soup.find_all(class_="jobsearch-JobMetadataHeader-iconLabel")])
        if 'Full-time' in temp and 'Part-time' in temp:
            indi_ftpt = 'Full-time / Part-time'
        elif 'Full-time' in temp:
            indi_ftpt = 'Full-time'
        elif 'Part-time' in temp:
            indi_ftpt = 'Part-time'
        elif 'Temporary' in temp:
            indi_ftpt = 'Temporary'
        else:
            indi_ftpt = np.nan      
    except:
        indi_ftpt = np.nan
    
    # find working location
    try: 
        indi_location = indi_soup.find_all('span',class_="jobsearch-JobMetadataHeader-iconLabel")[0].text
    except:
        indi_location = np.nan
    #find the posting time
    try:
        temp_time = indi_soup.find('div',class_="jobsearch-JobMetadataFooter").text
        indi_time = re.findall('Just posted|Today|\d+',temp_time)[-1]
        if indi_time in ['Just posted','Today']:
            indi_time = '0'
    except:
        indi_time = np.nan
    
    try:
        indi_desc = indi_soup.find('div',id='jobDescriptionText').text
    except:
        indi_desc = np.nan
    
    
   # to find the salary by joining all related html as a string
    temp_salary = "".join([str(i) for i in indi_soup.find_all(class_="jobsearch-JobMetadataHeader-itemWithIcon icl-u-textColor--secondary icl-u-xs-mt--xs")])
    salary_str = re.findall('\$.*0',temp_salary)
    if salary_str:
        indi_salary = salary_str[0]
    else:
        indi_salary = np.nan
           
    # find the popularity
    try:
        temp_review = indi_soup.find(class_="icl-Ratings-starsCountWrapper")['aria-label']
        indi_review = re.findall(r"\d[.]*\d*",str(temp_review))[0]
    except:
        indi_review = np.nan
    # find the number of review
    try:
        indi_review_num = indi_soup.find(class_="icl-Ratings-count").text.replace(' reviews','')
    except:
        indi_review_num = np.nan
    
    # Add all results into related list
    title.append(indi_title)
    company.append(indi_company)
    ftpt.append(indi_ftpt)
    location.append(indi_location)
    issued_time.append(indi_time)
    description.append(indi_desc)
    salary.append(indi_salary)
    review.append(indi_review)
    review_num.append(indi_review_num)
    
    #optional: for monitoring the progress
    cnt+=1
    print(cnt)
    
    
      

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [92]:
# add a new column for loacation comparasion
Country = ['Singapore' for i in range(len(list(href.values())))]
df = pd.DataFrame(data={'Issued time':issued_time,'Country':Country,'Job category': list(href.values()),'Job title':title,'Company':company,'Part time/ Full Time': ftpt,'Star review':review,'Number of reviews': review_num,'Location':location,'Job description': description,'Salary':salary, 'Indeed link': list(href.keys())})

In [93]:
# sort data in ascending order
df['Issued time'] = df['Issued time'].astype(int)
df.sort_values('Issued time', axis = 0, ascending = True, inplace=True)
df = df.drop_duplicates(['Job title','Company'],keep ='first')

In [94]:
removed_ls = ['Support','Programmer','Intern','Executive Assistant']
df = df[~df['Job title'].isin(removed_ls)]

In [95]:
def issue_time(x):
    x = str(x)
    if ' days ago' in x:
        modified = x.replace('days ago','')
    elif 'day ago' in x:
        modified = x.replace(' day ago','')
    elif 'Today' in x or 'Just posted' in x:
        modified = '0'
    else:
        modified = x
    if '+' in x:
        modified = x.replace('+',"")
    return modified
df['Issued time'] = df['Issued time'].apply(issue_time)

In [96]:
def min_max_salary(x):
    value = re.findall('((\d+)(k|K|m|M)*(\d*\.\d*)*(k|K|m|M))|((\d+,d+)-(d*,*d*))|(\d+,\d+)-*(\d*,*\d*)',x)     
    if len(value) == 0:
        min_result = np.nan
        max_result = np.nan
        
    elif len(value) >= 1:
        temp_result = []
        final_result = []
        
        # to remove empty string
        for tpl in value:
            indi_tpl = list(filter(lambda x: x !='', list(tpl)))
            temp_result.append(indi_tpl)
            
        for numls in temp_result:
            # to convert string into number
            if any(j in ['K','k'] for j in numls[0]):
                num = numls[0].replace('K','')
                num = num.replace('k','')
                num = float(num) * 1000
            elif any(j in ['M','m'] for j in numls[0]):
                num = numls[0].replace('M','')
                num = num.replace('m','')
                num = float(num) * 1000000
            elif ',' in numls[0]:
                num = float(''.join(numls[0].split(',')))
            else:
                num = np.nan
            
            final_result.append(num)
        if len(value) == 1:
            min_result, max_result = final_result[0], final_result[0]
        else:
            min_result = final_result[0]
            max_result = final_result[1]
    return [min_result, max_result]


df['Title salary'] = df['Job title'].apply(min_max_salary)
df['Min title salary']=df['Title salary'].apply(lambda x: x[0])
df['Max title salary']=df['Title salary'].apply(lambda x: x[1])

In [97]:
def min_max_salary_column(x):
    try:
        salary_str = str(x).replace('$','').replace(',','')

        if '-' in salary_str:
            numls = salary_str.split('-')
            min_salary = int(numls[0])
            max_salary = int(numls[1])
        else:
            min_salary, max_salary = int(salary_str),int(salary_str)
    except:
        min_salary, max_salary = np.nan, np.nan
    
    return [min_salary,max_salary]

df['Salary'] =  df['Salary'].apply(min_max_salary_column) 
df['Min header salary'] = df['Salary'].apply(lambda x: x[0])
df['Max header salary'] = df['Salary'].apply(lambda x: x[1])

In [98]:
df['Min salary'] = df['Min header salary'].fillna(df['Min title salary'])
df['Max salary'] = df['Max header salary'].fillna(df['Max title salary'])
df['Min salary'] = df['Min salary'].apply(lambda x:x*5.42 )
df['Max salary'] = df['Max salary'].apply(lambda x:x*5.42 )
df['Average salary'] = df.loc[:,'Min salary':'Max salary'].mean(axis=1)

def salary_range(x):
    if x < 10000:
        group = 'Below 10000'
    elif 10000 <= x < 20000:
        group = '10000-19999'
    elif 20000 <= x < 30000:
        group = '20000-29999'
    elif 30000 <= x < 40000:
        group = '30000-39999'
    elif 40000 <= x < 50000:
        group = '40000-49999'
    elif 50000 <= x < 60000:
        group = '50000-59999'
    elif 60000 <= x < 70000:
        group = '60000-69999'
    elif x>= 70000:
        group = '70000 or above'
    else:
        group = 'Not Mentioned'
    return group
    
df['Salary group'] = df['Average salary'].apply(salary_range)    
df = df.drop(['Salary','Title salary','Min header salary','Max header salary','Min title salary','Max title salary'],axis=1)

df

,Issued time,Country,Job category,Job title,Company,Part time/ Full Time,Star review,Number of reviews,Location,Job description,Indeed link,Min salary,Max salary,Average salary,Salary group
333,0,Singapore,Data analyst,Digital Content Marketing Specialist,CLICK2VIEW PTE. LTD.,NaN,NaN,NaN,Shenton Way,Roles & Responsibilities\nAttitude\n\nCuriosit...,https://sg.indeed.com/rc/clk?jk=0fa0b38f5ca4eb...,16260.00,24390.0,20325.00,20000-29999
766,0,Singapore,Data scientist,"Assistant Manager, Data & Innovation",AXA INSURANCE PTE. LTD.,NaN,3.9,"3,713",Shenton Way,Roles & Responsibilities\nAs a Senior Data Sci...,https://sg.indeed.com/rc/clk?jk=f5811dc3384d32...,35230.00,51490.0,43360.00,40000-49999
147,0,Singapore,Data analyst,Business Analyst,NTT DATA SERVICES SINGAPORE PTE. LTD.,Full-time,3.5,"2,436",Singapore,Roles & Responsibilities\nShould have worked i...,https://sg.indeed.com/rc/clk?jk=9686fc59aeb4df...,35230.00,51490.0,43360.00,40000-49999
252,0,Singapore,Data analyst,Equity Research (Emerging Markets),Ambition,NaN,4.1,15,Singapore,My client is a top tier fund in Singapore and ...,https://sg.indeed.com/rc/clk?jk=57ea426676fea9...,NaN,NaN,NaN,Not Mentioned
770,0,Singapore,Data scientist,Associate Product Owner,Yara,Full-time,NaN,NaN,Singapore,"Date posted: Apr 9, 2020\nLocation:\nSingapore...",https://sg.indeed.com/rc/clk?jk=9f7e881f022792...,NaN,NaN,NaN,Not Mentioned
976,0,Singapore,Data scientist,"Legal Counsel, Direct Sales",Autodesk,NaN,4.1,433,Singapore,This role requires international commercial tr...,https://sg.indeed.com/rc/clk?jk=4e4725ef984625...,NaN,NaN,NaN,Not Mentioned
748,0,Singapore,Data scientist,Chief Data Scientist,NextWave Partners,NaN,NaN,NaN,Singapore,AI Digital Product\nHands-on Technical Leader ...,https://sg.indeed.com/rc/clk?jk=0daf33e79bde6d...,NaN,NaN,NaN,Not Mentioned
749,0,Singapore,Data scientist,AI Data Scientist,17MEDIA SG PTE. LTD.,Full-time,NaN,NaN,Singapore,"Roles & Responsibilities\nAt 17 Media SG, we r...",https://sg.indeed.com/rc/clk?jk=644c181d6be035...,32520.00,54200.0,43360.00,40000-49999
1137,0,Singapore,Data scientist,Manager (Strategic Development),National Gallery Singapore,NaN,4.2,9,Singapore,ABOUT NATIONAL GALLERY SINGAPORE\nUNVEILING MO...,https://sg.indeed.com/rc/clk?jk=72c25e76039d52...,NaN,NaN,NaN,Not Mentioned
37,0,Singapore,Data analyst,Customer Analyst (Fairprice),STAFFONDEMAND PTE. LTD.,NaN,NaN,NaN,Boon Lay,Roles & Responsibilities\nCOMPANY DESCRIPTION\...,https://sg.indeed.com/rc/clk?jk=9df72d300a4c13...,24390.00,32520.0,28455.00,20000-29999


limit_output extension: Maximum message size of 10000 exceeded with 567583 characters

In [99]:
def min_edu_lv(x):
    x = str(x)
    if 'Diploma' in x:
        edu_lv = 'Diploma'
        
    elif ' hd ' in x.lower() or 'higher diploma' in x.lower():
        edu_lv = 'Higher Diploma'
    
    elif 'bachelor' in x.lower() or 'university' in x.lower() or 'degree holder' in x.lower() or 'Degree' in x or 'Bsc' in x:
        edu_lv = 'Bachelor'
        
    elif "master's" in x.lower() or 'MBA' in x or 'Msc' in x or 'Master' in x:
        edu_lv = "Master"
        
    elif 'phd' in x.lower() or 'doctorate' in x.lower():
        edu_lv = 'PhD'
        
    else:
        edu_lv = 'Not specified'

    return edu_lv

def max_edu_lv(x):
    x = str(x)
    if 'phd' in x.lower() or 'doctorate' in x.lower():
        edu_lv = 'PhD'
        
    elif "master's" in x.lower() or 'MBA' in x or 'Msc' in x or 'Master' in x:
        edu_lv = "Master"
    
    elif 'bachelor' in x.lower() or 'university' in x.lower() or 'degree holder' in x.lower() or 'Degree' in x:
        edu_lv = 'Bachelor'
        
    elif ' hd ' in x.lower() or 'higher diploma' in x.lower():
        edu_lv = 'Higher Diploma'
        
    elif 'Diploma' in x:
        edu_lv = 'Diploma'
    else:
        edu_lv = 'Not specified'

    return edu_lv
        

df['Minimum education level'] = df['Job description'].apply(min_edu_lv)
df['Maximum education level'] = df['Job description'].apply(max_edu_lv)    

    

In [100]:
skill_ls = {'Python': ['Python','Python'.lower()], 
             'Machine Learning':['Machine Learning'.lower(), 'Machine Learning'],
             'JavaScript': ['JavaSript'.lower(),'JavaScript'],
             'R':[' R '],
             'Java':['Java ','Java'.lower(),' Java '],
             'IoT':['IoT'],
             'Excel':['Microsoft Excel','Excel',' Excel '], 
             'Power BI':['BI','Power BI'],
             'English':['English','english'],
             'Chinese':['Cantonese','Cantonese'.lower(),'Mandarin','Mandarin'.lower(),'Chinese','Chinese'.lower(),"Putonghua",'putonghua'],
             'SAS':['SAS','sas'.lower()],
             'Oracle':['Oracle','Oracle'.lower()],
             'C++':['C++','c++'],
             'C#':['C#','c#'],
             'C':[' C ',' c '],
             'Qlik':['Qlik','Qlik'.lower()],
             'alteryx':['Alteryx','Altery'],
             'Tableau':['tableau','Tableau'],
             'SPSS':['SPSS','SPSS'.lower()],
             'Powerpoint':['Microsoft Powerpoint','Microsoft Powerpoint'.lower(),'Powerpoint','powerpoint'.lower()],
             'PostgreSQL':['PostgreSQL','PostgreSQL'.lower()],
             'SQL':[' SQL ',' SQL '.lower()],
             'MySQL':['MySQL','MySQL'.lower()]}    

# Summarize skills into two categories
program_ls = []
visual_ls = []
for ls in [skill_ls['Python'],skill_ls['R']]:
    for j in ls:
        program_ls.append(j)
for ls in [skill_ls['Tableau'], skill_ls['Qlik'], skill_ls['Power BI']]:
    for j in ls:
        visual_ls.append(j)
skill_ls['Visualisation tool'] = visual_ls
skill_ls['Programming tool'] = program_ls
# append to column
for key, value in skill_ls.items():
    df[key] = df['Job description'].apply(lambda x: 'Y'  if any(j in x for j in value) else np.nan)









        

In [101]:
df

Issued time    Country    Job category  \
333            0  Singapore    Data analyst   
766            0  Singapore  Data scientist   
147            0  Singapore    Data analyst   
252            0  Singapore    Data analyst   
770            0  Singapore  Data scientist   
976            0  Singapore  Data scientist   
748            0  Singapore  Data scientist   
749            0  Singapore  Data scientist   
1137           0  Singapore  Data scientist   
37             0  Singapore    Data analyst   
408            0  Singapore    Data analyst   
410            0  Singapore    Data analyst   
413            0  Singapore    Data analyst   
41             0  Singapore    Data analyst   
750            0  Singapore  Data scientist   
43             0  Singapore    Data analyst   
51             0  Singapore    Data analyst   
1096           0  Singapore  Data scientist   
221            0  Singapore    Data analyst   
20             0  Singapore    Data analyst   
469            0  Singapore    Data analyst   
486            0  Singapore    Data analyst   
127            0  Singapore    Data analyst   
93             0  Singapore  Data scientist   
3              0  Singapore    Data analyst   
5              0  Singapore    Data analyst   
648            0  Singapore    Data analyst   
7              0  Singapore    Data analyst   
740            0  Singapore    Data analyst   
65             0  Singapore    Data analyst   
11             0  Singapore    Data analyst   
1019           0  Singapore  Data scientist   
308            1  Singapore    Data analyst   
398            1  Singapore    Data analyst   
505            1  Singapore    Data analyst   
84             1  Singapore    Data analyst   
890            1  Singapore  Data scientist   
383            1  Singapore    Data analyst   
73             1  Singapore    Data analyst   
550            1  Singapore    Data analyst   
132            1  Singapore    Data analyst   
130            1  Singapore    Data analyst   
381            1  Singapore    Data analyst   
61             1  Singapore    Data analyst   
62             1  Singapore    Data analyst   
117            1  Singapore    Data analyst   
405            1  Singapore    Data analyst   
76             1  Singapore    Data analyst   
195            1  Singapore    Data analyst   
397            1  Singapore    Data analyst   
89             1  Singapore    Data analyst   
775            1  Singapore  Data scientist   
4              1  Singapore    Data analyst   
209            1  Singapore    Data analyst   
183            1  Singapore    Data analyst   
343            1  Singapore    Data analyst   
13             1  Singapore    Data analyst   
360            1  Singapore    Data analyst   
214            1  Singapore    Data analyst   
537            1  Singapore    Data analyst   
16             1  Singapore    Data analyst   
169            1  Singapore    Data analyst   
164            1  Singapore    Data analyst   
874            1  Singapore  Data scientist   
152            1  Singapore    Data analyst   
467            1  Singapore    Data analyst   
27             1  Singapore    Data analyst   
170            2  Singapore    Data analyst   
758            2  Singapore  Data scientist   
922            2  Singapore  Data scientist   
141            2  Singapore    Data analyst   
920            2  Singapore  Data scientist   
901            2  Singapore  Data scientist   
489            2  Singapore    Data analyst   
103            2  Singapore  Data scientist   
106            2  Singapore    Data analyst   
587            2  Singapore    Data analyst   
760            2  Singapore  Data scientist   
668            2  Singapore    Data analyst   
266            2  Singapore    Data analyst   
182            2  Singapore    Data analyst   
456            2  Singapore    Data analyst   
114            2  Singapore    Data analyst   
848            2  Singapore  Data scientist   
260       

limit_output extension: Maximum message size of 10000 exceeded with 1164233 characters

In [102]:
df.to_csv('Indeed4.csv')

In [37]:
pwd

'/Users/dick/Downloads'